In [1]:
import cv2

import sys
sys.path.insert(0, '..')

from decompress import upscale, interpolate_recursively, Interpolator

import numpy as np
import tensorflow as tf
import pickle

import mediapy as media

In [47]:
'''
    Decoding .mlpg file
'''
path = '../frames'

with open(f'{path}/data.mlpg', 'rb') as f:
    data = pickle.load(f)

mlpg_frames = [cv2.cvtColor(cv2.imdecode(
    np.frombuffer(frame.read(), np.uint8), 
    cv2.IMREAD_COLOR), cv2.COLOR_BGR2RGB)
    for frame in data['keyframes']]

print(f'video with {len(mlpg_frames)} frames')
media.show_video(mlpg_frames, fps=15)

video with 19 frames


This browser does not support the video tag.


In [7]:
'''
    Normalizing frames to float32
'''
normalized_float32_mlpg_frames = [(frame).astype(np.float32) / 255.0
                                    for frame in mlpg_frames]
media.show_video(normalized_float32_mlpg_frames, fps=1)

This browser does not support the video tag.


In [8]:
'''
    Interpolating frames
'''
print('Converting')
tf_input_frames = tf.convert_to_tensor(normalized_float32_mlpg_frames, np.float32)

print('Creating interpolator')
interpolator = Interpolator()
times_to_interpolate = 3

print('Interpolating frames')
interpolated_frames = list(interpolate_recursively(tf_input_frames, times_to_interpolate,
                                        interpolator))

Converting
Creating interpolator
Interpolating frames


In [42]:
'''
    Convert back from Tensor
'''

interpolated_numpy_frames = [np.array(frame) 
                             for frame in interpolated_frames]

print(f'video with {len(interpolated_numpy_frames)} frames')
media.show_video(interpolated_numpy_frames, fps=30, title='interpolated video')

145
270
480
3
[0.98039216 0.9254902  0.92941177]


AxisError: source: axis -1 is out of bounds for array of dimension 0

In [6]:
'''
    Writing Video with mediapy
'''
upscaled_frames = upscale(media.to_uint8(interpolated_numpy_frames), '../models/')
media.write_video('output.mp4', upscaled_frames, fps=30)